# Benzinga-Nachrichten-Verarbeitung

## Setup up

### Cluster spin up

In [1]:
%load_ext autoreload
%autoreload 2
import sys, io, tarfile, os
import subprocess
from src.cloud_manager import spin_up_cluster, fn_to_targz_string
from distributed.diagnostics.plugin import WorkerPlugin
from dask.distributed import Client
import dask
###
import dask.dataframe as dd
import pandas as pd
from src.preprocessing.news_parser import get_company_abbreviation, yahoo_get_wrapper, \
                                          infer_author, filter_body
# import nltk

# Mainly for topic modeling
# from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
# import gcsfs
import re

In [3]:
cluster = spin_up_cluster()

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Task exception was never retrieved
future: <Task finished name='Task-90' coro=<PortForward._sync_sockets() done, defined at 
/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a 
TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
    |     raise ConnectionClosedError("TCP socket closed")
    | kr8s._exceptions.ConnectionClosedError: TCP socket closed
    +------------------------------------

Task exception was never retrieved
future: <Task finished name='Task-87' coro=<PortForward._sync_sockets() done, defined at 
/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a 
TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
    |     raise ConnectionClosedError("TCP socket closed")
    | kr8s._exceptions.ConnectionClosedError: TCP socket closed
    +------------------------------------

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [4]:
# Cant extract this out of the notebook as then import would be missing...
def extract_targz_string(s,*args,**kwargs):
    with io.BytesIO() as bt:
        bt.write(s) 
        bt.seek(0)
        with tarfile.open(fileobj=bt,mode='r:gz') as tf:
            tf.extractall(*args,**kwargs)

class SourceDirectoryCopier(WorkerPlugin):
    def __init__(self, targz):
      self.targz = targz
      super().__init__()

    def setup(self, worker=None, **kwargs):
        self.worker = worker
        for elem in self.targz:
            extract_targz_string(elem)
        sys.path.append("/")
        print("Added a new worker at:", worker)
        subprocess.run([sys.executable, "-m", "pip", "install", "loky"], check=True)


targz = [fn_to_targz_string("src"), fn_to_targz_string("extreme-lore-398917-ac46de419eb2.json")]
plugin = SourceDirectoryCopier(targz)
client: Client = Client(cluster)
client.run_on_scheduler(plugin.setup)
client.register_plugin(plugin)

{}

In [4]:
# print(cluster.dashboard_link)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_kubernetes.KubeCluster
Dashboard: http://localhost:54347/status,
Dashboard: http://localhost:54347/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.0.22:8786,Workers: 0
Dashboard: http://10.12.0.22:8787/status,Total threads: 0
Started: 2 minutes ago,Total memory: 0 B


In [6]:
client.run_on_scheduler(os.environ.get, "GOOGLE_APPLICATION_CREDENTIALS")

'/extreme-lore-398917-ac46de419eb2.json'

In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "extreme-lore-398917-ac46de419eb2.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'extreme-lore-398917-ac46de419eb2.json'

In [17]:
from os import listdir

In [19]:
client.run_on_scheduler(listdir, "/")

['proc',
 'home',
 'lib',
 'opt',
 'run',
 'bin',
 'root',
 'etc',
 'boot',
 'sbin',
 'sys',
 'srv',
 'lib32',
 'tmp',
 'lib64',
 'mnt',
 'dev',
 'libx32',
 'usr',
 'var',
 'media',
 'src',
 'extreme-lore-398917-ac46de419eb2.json',
 'requirements_google.txt',
 'requirements_clean.txt']

### Source File Copy Preparation

In [ ]:
# client.run_on_scheduler(lambda x: sys.path, x=2)
# client.run_on_scheduler(subprocess.run, args='export PYTHONPATH=$PYTHONPATH:', shell=True)
# client.run_on_scheduler(subprocess.run, args='echo "', shell=True)

In [ ]:
# def myfunc():
#     from src.preprocessing.news_parser import filter_body
#     return filter_body

# client.run(myfunc)
# client.run_on_scheduler(myfunc)
# x = client.submit(myfunc)
# x.result()

# client.run_on_scheduler()

### System settings

In [ ]:
using_colab = False
using_laptop = True
import os

%%capture
if using_colab:
  from google.colab import drive
  drive.mount('/content/drive')

### CUDF für hardware acceleration

In [ ]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
# import cudf

### Imports

In [ ]:
%%capture
if using_colab:
  !sudo apt update
  !sudo apt install maven;

  !pip install pyarrow==11.0.0
  !pip install html2text
  !pip install datefinder
  !pip install -U dask[complete]
  !pip install nltk
  !pip install dateparser
  !pip install pyngrok
  !pip install sutime
  !pip install pyngrok

  # This is required for sutime
  !mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")');

### Setup NGROK tunnel

In [ ]:
# ------- Only required when working on google colab, not required when working with google cloud engine -----------------
if using_colab:
  from pyngrok import ngrok, conf
  conf.get_default().auth_token = "2WntwErWDt9LxQ2Jfp6C8OxDAMK_7iZVdC1utyZET1PE8cuUg"

  public_url = ngrok.connect(8787).public_url
  print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, 8787))

In [ ]:
%%capture
if 'client' not in globals():
  !python -m pip install jupyter-server-proxy
  client = Client()

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/unraw2_bzg/'
output_dir = cwd+'/data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

In [ ]:
auhtor_value_counts.sum().diff()

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Vorgehen

Benötigt:
---------
- data/tickers.pkl (alt) \\
- data_shared/corporation_endings.txt \\
- input_dir = data/unraw3_bzg/ \\
- output_dir = data/unraw4_bzg/

Produziert:
-----------
- data_shared/all_ticker_name_mapper.parquet

--------------

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt! Bzw. generell ist das eine gute Datensäuberungs-Maßnahme.

In [ ]:
input_dir = cwd+'/data/unraw3_bzg/'
output_dir = cwd+'/data/unraw4_bzg/'
ddf = dd.read_parquet(input_dir)

In [ ]:
all_tickers = ddf.stocks.unique().compute()

### Full-Name-Discovery

In [ ]:
company_names = all_tickers.apply(lambda x: yahoo_get_wrapper(x))

In [ ]:
all_mapper = pd.concat([all_tickers, company_names], axis=1)

In [ ]:
all_mapper.columns = ["ticker", "company_name"]

In [ ]:
all_mapper = all_mapper.dropna() # This drops like half of the tickers.

In [ ]:
all_mapper[all_mapper.company_name.apply(lambda x: "Alphabet" in x)]

In [ ]:
print(len(all_mapper))
print(len(all_mapper.company_name.unique()))

In [ ]:
vcs = all_mapper.company_name.value_counts()
vcs = vcs[vcs >= 2]

In [ ]:
# Ticker-Grouping
all_mapper[all_mapper.company_name.isin(vcs.index)].sort_values("company_name")

Es ist nicht leicht zu sagen, welchen von den Tickern wir bevorzugen sollten. Abgleichen mit den Aktientickern des Kursdatensatzes notwendig, um zu sehen, ob überhaupt nur ein Ticker übereinstimmt. Wenn es für beide Ticker eine Kurszeitreihe gibt, dann sollten wir die nehmen, die ein höheres historisches Volumen hat. Dies ist allerdings etwas, was wir später machen und nicht jetzt. Hier wollen wir zunächst nur die Nachrichten verarbeiten, weswegen wir nur die NaN-Unternehmen rausnehmen und den Rest - *ohne Ticker-Filtering* - weiterverarbeiten.

In [ ]:
mapper = all_mapper

In [ ]:
mapper.columns = ["ticker", "company_names"]
mapper = mapper[mapper.isna().sum(axis=1) == 0]

In [ ]:
mapper = mapper.set_index("ticker")

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
# Apply get_company_abbreviation twice in order to get rid of Enterprise, Ltd.
# Otherwise , Ltd. remains.
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x, company_endings=company_endings))

In [ ]:
print(mapper.short_name.isna().sum()) # 2037 stocks for which we don't have an ending to abbreviate
# mapper.loc[:, "short_name"] = mapper.short_name.fillna(mapper.company_names)
mapper = mapper.applymap(lambda x: x.strip(" "))

In [ ]:
mapper.to_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
mapper = pd.read_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [ ]:
ddf.shape[0].compute()

In [ ]:
filt_ddf.shape[0].compute()

Es verbleiben circa 1 Mio. Nachrichten, für die wir den Ticker zu einem FIrmennamen auflösen können.

Diese Nachrichten können wir nun wirklich parsen, und danach ordentlich kategorisieren.

In [ ]:
ddf = filt_ddf
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# TODO: Kann effizienter werden mit GroupBy-Operation
ddf["company_name"] = ddf.stocks.apply(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.apply(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest/', name_function=name_function)

### Firmennamen-Nachrichtenkörper-Verifikation

In [ ]:
ddf = dd.read_parquet(cwd+'/data/latest/')

In [ ]:
# ddf["time"] = ddf["time"].astype(str)

In [ ]:
def f(df):
    df["time"] = df["time"].astype(str)
    return df
# futures = client.submit(f, ddf)
# ddf = ddf.map_partitions(f)

In [ ]:
# results = client.gather(futures)

In [ ]:
mask = ddf.apply(lambda x: bool(re.search(x["short_name"], x["body"].replace("\n", " "), re.IGNORECASE)) or \
                 bool(re.search(x["short_name"], x.title, re.IGNORECASE)),
                 axis=1,
                 meta=pd.Series(dtype=bool))

In [ ]:
# Around 11k stocks before `filtering`
# len(ddf.stocks.unique())

# Around 10k stocks after `filtering`
# len(ddf[mask].stocks.unique())

In [ ]:
ddf[mask].shape[0].compute()

In [ ]:
# Case 1: Firmenname wird in abgekürzter Version benutzt, die wir nicht methodisch rekonstruieren können
#   Z.B.: IBM -> International Business Machines,
#   oder UPS -> United Parcel Service
#   oder GE -> General Electric
# Case 2: Ticker wurde recycled und zeigt inzwischen auf eine andere Firma.
# Will hierzu nicht auch noch Daten kaufen, deswegen werden hier leider einige Nachrichten verworfen werden.

ddf[~mask].shape[0].compute() # ~120k Nachrichten mit `FALSCHEM` Firmennamen -> Textinhalt ableichen mit Firmennamen des Kursdaten-Datensatzes.

In [ ]:
# ddf[~mask].stocks.value_counts().compute().head(15)

In [ ]:
# Filter for high conviction entries
ddf = ddf[mask]

In [ ]:
ddf = ddf.repartition(npartitions=30)

In [ ]:
ddf.head()

### Duplikate Entfernen

In [ ]:
# import cudf

In [ ]:
# ddf = ddf.map_partitions(cudf.from_pandas)

In [ ]:
res = ddf.map_partitions(lambda x: x.drop_duplicates())

In [ ]:
# res.shape[0].compute()

In [ ]:
ddf = res

In [ ]:
ddf.head()

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest2/', name_function=name_function)

#### Convert ddf to pd.DataFrame

In [ ]:
#ddf = ddf.compute()
ddf = ddf.sort_values("time")

### Timedeltas zwischen Nachrichtenmeldungen



Wir sehen, dass einige Nachrichten dupliziert vorkommen, d.h. mit einem Timedelta von 0 und mit derselben Überschrift etc. diese gilt es zu eliminieren.

In [ ]:
tmp = ddf[["time", "stocks"]]

In [ ]:
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())

In [ ]:
# ~3 minutes evaluates to true
# (news_timedeltas.index == ddf.index).all()

In [ ]:
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta

In [ ]:
news_timedeltas.iloc[0].components

In [ ]:
same_day_timedeltas = news_timedeltas.apply(lambda x: x.components.days == 0)

In [ ]:
(same_day_timedeltas == 0).sum()

In [ ]:
same_hour_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0))

In [ ]:
print(same_hour_timedeltas.sum())

In [ ]:
same_minute_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0) & \
                                               (x.components.minutes == 0))

In [ ]:
print(same_minute_timedeltas.sum())

In [ ]:
same_day_ddf = ddf.loc[same_day_timedeltas]

In [ ]:
ddf.stocks.value_counts().describe()

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert.

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Topic Modeling (Nach Aschluss Auslagern in Python file)

In [ ]:
# Scaling up cluster and installing required packages
cluster.scale(5)
client.wait_for_workers(5)
client.run(worker_setup)

In [ ]:
ddf = dd.read_parquet("gcs://extreme-lore-398917-bzg/latest2/",
                      storage_options={'token': token})

In [ ]:
# Remove numbers
docs = ddf.body.map(lambda x: re.sub(r'\d+', '', x), meta=pd.Series(dtype="string"))

In [ ]:
docs.name="body"

### TF-IDF

In [ ]:
from src.topic_modeling.helpers import tokenize, keyword_filter
import numpy as np

In [ ]:
keywords_list1 = ["launch", "business", "strategy", "management", "product", "service", "app", "customer", "merge"]
keywords_list2 = ["upgrade", "downgrade", "raise", "cut", "buy", "sell", "hold", "outperform", "underperform", "analyst", "estimate"]
keywords_list3 = ["ebit", "eps", "earnings", "report", "financial", "quarter", "annual", "year", "ended", "net", "income"]
total_keywords = keywords_list1 + keywords_list2 + keywords_list3
len(total_keywords)

In [ ]:
# Tokenization + rough filtering
tfidf_docs = docs.map(lambda x: keyword_filter(tokenize(x), total_keywords), meta=pd.Series(dtype="object"))
tfidf_docs = tfidf_docs.reset_index().repartition(npartitions=5)
tfidf_docs.columns = ["index", "body"]
tfidf_docs.to_parquet("gcs://extreme-lore-398917-bzg/tfidf-tokens",
                                                    storage_options={'token': token})

In [ ]:
tfidf_docs = dd.read_parquet("gcs://extreme-lore-398917-bzg/tfidf-tokens/",
                                                    storage_options={'token': token},
                                                    dtype_backend="pyarrow")

In [ ]:
tfidf_docs = tfidf_docs.compute()

In [ ]:
# Numpy representation is without commas making us unable to convert strings to list via eval
tfidf_docs.loc[:, "body"] = tfidf_docs.body.map(lambda x: x.replace(" ", ", "))

In [ ]:
def f(tokens):
    if len(tokens) == 0:
        return np.array(["wordtopreventemptydocumentswhencalculatingtfidf"])
    else:
        return np.array(tokens)
        
tfidf_docs.loc[:, "body"] = tfidf_docs.body.map(lambda x: f(eval(x)))

In [ ]:
tfidf_docs.iloc[0].body

In [ ]:
topic_tfidfs_list = []
for topic_tokenizer in [keywords1, keywords2, keywords3]:
    lazy_result = dask.delayed(TfidfVectorizer(tokenizer=topic_tokenizer, lowercase=False).fit_transform)(tfidf_docs.body)
    lazy_sums = dask.delayed(np.apply_along_axis)(np.sum, 1, lazy_result.todense())
    topic_tfidfs_list.append(lazy_sums)

In [ ]:
topic_indicator_list = dask.compute(*topic_tfidfs_list)

In [ ]:
df = pd.DataFrame(topic_indicator_list)
df=df.transpose()

In [ ]:
tfidf_docs.head().iloc[4].body

In [ ]:
df.head()

### LDA

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import Phrases
import nltk

In [ ]:
# Split the documents into tokens.
lda_docs = docs.apply(lambda x: word_tokenize(str.lower(x)), meta=pd.Series(dtype="object"))

In [ ]:
# Remove words that are only one character.
# Lemmatize the documents.
lemmatizer = WordNetLemmatizer()
lda_docs = lda_docs.apply(lambda x: [lemmatizer.lemmatize(token) for token in x if len(token) >1], meta=pd.Series(dtype="object"))

In [ ]:
lda_docs.head()

In [ ]:
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
def f(doc):
  doc = doc
  for token in bigram[doc]:
      if '_' in token:
          # Token is a bigram, add to document.
          doc.append(token)
  return doc
lda_docs = lda_docs.apply(lambda doc: f(doc))

In [ ]:
# lda_docs.name = "body"
# lda_docs = lda_docs.to_frame().reset_index()
# lda_docs = lda_docs.reset_index().repartition()

In [ ]:
# lda_docs.to_parquet(cwd + "/data/lda_docs", schema={"body": pa.string(),  "index":pa.int32()})
# lda_docs = dd.read_parquet(cwd + "/data/lda_docs")

In [ ]:
# Remove rare and common tokens.

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=1000, no_above=0.6)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [ ]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

## Nachrichten-Parsing


In [ ]:
# dask.config.set({'distributed.scheduler.worker-ttl': "60 minutes"})

In [8]:
# Wait an appropriate amount of time to give autoscaler opportunity to create nodes.
# Even CommClosedError occurrs... just wait
n = 2
# cluster.scale(n, worker_group="highmem")
client.wait_for_workers(n)

In [ ]:
n = 1
# cluster.scale(0, worker_group="highmem-single")
client.wait_for_workers(n) 

In [9]:
ddf = dd.read_parquet("gcs://extreme-lore-398917-bzg/latest2/")
                    #   storage_options={'token': token})

### Beispiel/ Untersuchung

In [ ]:
sample_partition = ddf.get_partition(20)
y = sample_partition.head(5)
y.loc[:, "time"] = pd.to_datetime(y.time).dt.tz_convert("UTC")
x = y.iloc[2]
x.body

In [ ]:
res = client.submit(filter_body, row=x, logging=False)

In [ ]:
res.result()

### Anwenden der filter_body-Funktion auf alle Reihen:

In [10]:
def handle_timezone(x):
    try:
        return x.tz_convert("US/Eastern")
    except Exception as e:
        return x.tz_localize("US/Eastern")

In [11]:
# Still need to parse time correctly...
ddf["time"] = ddf["time"].map(lambda x: handle_timezone(pd.to_datetime(x)), meta=pd.Series(dtype="datetime64[ns, US/Eastern]"))

In [12]:
# part_n = 1
# par = ddf.partitions[:3]
par = ddf  
# par = ddf

In [13]:
par["parsed_body"] = par.map_partitions(lambda y: y.apply(lambda x: filter_body(x),
                                                axis=1),
                                                meta=pd.Series(dtype="string"))

In [14]:
# Hier der error letztes Mal
# from src.cloud_manager import VMManager

In [15]:
par = client.persist(par) 

Task exception was never retrieved
future: <Task finished name='Task-630' coro=<PortForward._sync_sockets() done, defined at /opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
    |     raise ConnectionClosedError("TCP socket closed")
    | kr8s._exceptions.Conn

In [28]:
client.nthreads()

{'tcp://10.12.1.3:38925': 3, 'tcp://10.12.1.4:45535': 3}

Task exception was never retrieved
future: <Task finished name='Task-2420' coro=<PortForward._sync_sockets() done, defined at /opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=HTTPStatusError("Client error '401 Unauthorized' for url 'https://34.70.147.190/api/'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401")>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 170, in _sync_sockets
    async with self._connect_websocket() as ws:
  File "/opt/conda/lib/python3.10/contextlib.py", line 199, in __aenter__
    return await anext(self.gen)
  File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 147, in _connect_websocket
    self.pod = await self._select_pod()
  File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 137, in _select_pod
    return random.choice(await self._resource.ready_pods())
  File "/opt/conda/lib/python3.1

In [ ]:
import time 
while(True):
    time.sleep(10)
    print(client.get_scheduler_logs())

Task exception was never retrieved
future: <Task finished name='Task-920' coro=<PortForward._sync_sockets() done, defined at /opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
    |     raise ConnectionClosedError("TCP socket closed")
    | kr8s._exceptions.Conn

In [ ]:
from dask.distributed import wait, progress

In [ ]:
# progress(par)

In [ ]:
wait(par)

In [ ]:
# Do this only after having persisted, because this blocks the console making us unable to interact witth client/cluster
# and can interfere with communication between client and scheduler.

# VMManager needs to be redone... do os.system(kubectl delete dsk example), its a cleaner shutdown and also deletes nodes
# with VMManager(cluster) as _:
par.to_parquet("gcs://extreme-lore-398917-bzg/processed_news",
        storage_options={'token': token})

In [ ]:
client # no connection, can we connect via ip??

In [ ]:
cluster # connection still here?

## Analyse der durschnittlichen Tokenlänge

## Generating timestamp-stock pairs in order to know for which days we need stock data